In [10]:
import re

class Morph():
    def __init__(self, doc_list):
        self.surface = doc_list[0]
        self.base = doc_list[7]
        self.pos = doc_list[1]
        self.pos1= doc_list[2]

    def __str__(self):
        return "{" + f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}" + "}"

class Chunk():
    def __init__(self,morph_list,dst,srcs):
        self.morph_list = morph_list
        self.dst = int(dst)
        self.srcs = srcs

    def __str__(self):
        morphs_str = ', '.join(str(morph) for morph in self.morph_list) # Morphオブジェクトのリストを文字列に変換
        return f"Morphs: [{morphs_str}], Dst: {self.dst}, Srcs: {self.srcs}"

# 文節番号と係り先番号の取得
def find_feature_number(text):
    pattern = r"^\* \d+ ([^\d]*\d+)D"
    match = re.search(pattern, text)
    if match:
        dst_number = match.group(1)  # 文節番号  # 係先番号
        return dst_number
    else:
        return None


In [11]:
with open(file="ai.ja.txt.parsed", mode="r", encoding="utf-8")as f:
    docs = f.readlines()

Chunk_list = []
chunk_flag = False
# srcの追加処理は後で行う
for doc in docs[5:]:
    if doc == "EOS\n":
        chunk = Chunk(morph_list,dst_num,[])
        Chunk_list.append(chunk)
        break
    # *から始まる行とEOSの行は飛ばす
    if doc[0]=="*":
        if chunk_flag:
            chunk = Chunk(morph_list,dst_num,[])
            Chunk_list.append(chunk)
        dst_num = find_feature_number(doc)
        morph_list = []
        chunk_flag = True
    else:
        doc = doc.replace("\t",",").replace("\n", "")
        doc_list = doc.split(",")
        doc_morph = Morph(doc_list)
        morph_list.append(doc_morph)


# Srcsを埋めていく
for i in range(len(Chunk_list)):
    Chunk_list[Chunk_list[i].dst].srcs.append(i)

In [15]:
# 41を再利用していく

# morphのsurfaceのみを抽出して，文章を再構築する
def get_sentence(chunk: Chunk):
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    morph_list = chunk.morph_list
    sentence_str = ""
    for morph in morph_list:
        if morph.surface in remove_list:
            continue
        sentence_str += morph.surface
    return sentence_str

for chunk in Chunk_list:
    src_dst_sentence = ""
    src_dst_sentence += get_sentence(chunk) + "\t"
    if chunk.dst != -1: #係先がない場合は係先を出力しない
        src_dst_sentence += get_sentence(Chunk_list[chunk.dst])
    print(src_dst_sentence)

人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される
される	


**リーダブルコード**  
- P16 具体的な名前を使う
- P19 単語に情報を追加する
- P51 コードを段落に分割する
- P100 説明変数の導入